# Regression PLots

[Resource](https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.formula.api import ols

plt.rc('figure', figsize=(16, 8))
plt.rc('font', size=14)